# StylePeople inference code to fit neural textures to images

Please download the archive with data from [Google Drive](https://drive.google.com/drive/folders/1-Rzn0fR4Rq2VKS7EUg8hCdJwFe4niNJr) under `drive_path/stylepeople_data`

To infer a neural texture for new data samples, run the following installations and execute the last cell

## Setup : mouting drive and installations

In [ ]:
### Mount google drive if available
try:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/master_thesis/'
    in_colab = True
except:
    drive_path = ''
    in_colab = False

In [ ]:
### Install all StylePeople's dependecies : see https://github.com/saic-vul/style-people/blob/main/docker/Dockerfile
import os

# Install stylepeople, rasterizer and requirements
!git clone https://github.com/saic-vul/style-people.git
!pip install git+https://github.com/rmbashirov/minimal_pytorch_rasterizer
!pip --no-cache-dir install -r style-people/requirements.txt

# Install conda
!wget --quiet https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh -O ~/miniconda.sh
!chmod +x ~/miniconda.sh
!~/miniconda.sh -b -p ~/miniconda && \
!rm ~/miniconda.sh && \
!conda clean -ya

!conda install pytorch torchvision cudatoolkit=10.1 -c pytorch

os.environ['PYTHONDONTWRITEBYTECODE'] = '1'
os.environ['PYTHONUNBUFFERED'] = '1'

# Install nvdiffrast plugin
!git clone https://github.com/NVlabs/nvdiffrast

!apt-get update && apt-get install -y \
    pkg-config \
    libglvnd0 \
    libgl1 \
    libglx0 \
    libegl1 \
    libgles2 \
    libglvnd-dev \
    libgl1-mesa-dev \
    libegl1-mesa-dev \
    libgles2-mesa-dev

os.environ['LD_LIBRARY_PATH'] = '/usr/lib64:$LD_LIBRARY_PATH'

os.environ['NVIDIA_VISIBLE_DEVICES'] = 'all'
os.environ['NVIDIA_DRIVER_CAPABILITIES'] = 'compute,utility,graphics'

os.environ['PYOPENGL_PLATFORM'] = 'egl'

!cp nvdiffrast/docker/10_nvidia.json /usr/share/glvnd/egl_vendor.d/10_nvidia.json

%cd nvdiffrast/
!pip install .
%cd ..

## Setup : adapt the code to our files paths

In [3]:
### Replace string with another one in a file
def replace_string(filename:str, to_replace:str, replacement:str):
    with open(filename, 'r') as f:
        filedata = f.read()

    filedata = filedata.replace(to_replace, replacement)

    with open(filename, 'w') as f:
        f.write(filedata)

    return

In [4]:
### Comment some lines in a file
from typing import List

def comment_lines(filename:str, replace_lines:List[int]):
    with open(filename, 'r') as f:
        lines = f.readlines()

    for n, line in enumerate(lines):
        for replace in replace_lines:
            if(n == replace - 1):
                commented_line = '#' + line
                lines[n] = commented_line

    with open(filename, 'w') as f:
        f.write(''.join(lines))

    return

In [5]:
### Replace 'data/' with 'stylepeople_data/' in all files : stylepeople_data/ = https://drive.google.com/drive/folders/1-Rzn0fR4Rq2VKS7EUg8hCdJwFe4niNJr
datapath1 = drive_path + 'stylepeople_data/'
datapath2 = drive_path + 'stylepeople_data'

file_list = ['inference_module/inferer.py', 'render_person.py', 'sample_new_textures.py', 'utils/demo.py', 'utils/uv_renderer.py', 'infer_texture.py']

for f in file_list:
    filename = 'style-people/' + f
    replace_string(filename, to_replace='data/', replacement=datapath1)

replace_string(filename='style-people/inference_module/config.yaml', to_replace='data', replacement=datapath2)
replace_string(filename='style-people/utils/smplx_models.py', to_replace='.pkl', replacement='.npz')
replace_string(filename='style-people/infer_texture.py', to_replace='inference_module/', replacement='style-people/inference_module/')
replace_string(filename='style-people/utils/demo.py', to_replace='/checkpoints/', replacement='/checkpoint/')
replace_string(filename='style-people/inference_module/inferer.py', to_replace='"checkpoints"', replacement='"checkpoint"')
replace_string(filename='style-people/inference_module/runner.py', to_replace='kornia.resize', replacement='kornia.geometry.transform.resize')

### In style-people/sample_new_textures.py, comment lines 15, 16, 17 and 19
comment_lines(filename='style-people/infer_texture.py', replace_lines=[14, 15, 16, 18])

In `style-people/inference_module/criterion.py`, add the code from the following cell after line 32

In [ ]:
'''
fct = 1 # sampling factor
while True:
    try:
        loss_not_reduced = self.lpips(pred.contiguous(), target.contiguous())
        break
    except RuntimeError:
        pred = pred[:,:,::fct,::fct]
        target = target[:,:,::fct,::fct]
        fct += 1
    if fct == 6:
        raise RuntimeError('CUDA out of memory.')
'''

## Infer a neural texture

In [ ]:
### Infer a neural texture for a given set of images

SMPLX_MODEL_PATH = drive_path + 'smplx'

# ZJU MoCap
SAVEPATH = drive_path + 'zju_mocap_1_frame/inference_data'
DATAPATH = drive_path + 'zju_mocap_1_frame/pickle_data'

# People Snapshot
'''
subject_path = drive_path + 'stylepeople_snapchot/people_snapshot_f1c/'
SAVEPATH = subject_path + 'inferstyle1'
DATAPATH = subject_path + 'styledata1'
'''

# Azure People
'''
subject = 'azure_01'
sampels_path = drive_path + 'stylepeople_data/inference_samples/'
SAVEPATH = drive_path + 'azure_inference/' + subject
DATAPATH = sampels_path + subject
'''

%run style-people/infer_texture.py --input_path $DATAPATH --texture_out_dir $SAVEPATH --smplx_model_dir $SMPLX_MODEL_PATH